<a href="https://colab.research.google.com/github/destroyer000lucky/Gymnasium/blob/main/ReplayBufferPruningCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# define a replay buffer
import collections
import typing

import numpy as np


_field_names = [
    "state",
    "action",
    "reward",
    "next_state",
    "done"
]
Experience = collections.namedtuple("Experience", field_names=_field_names)


class ExperienceReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self,
                 batch_size: int,
                 buffer_size: int = None,
                 random_state: np.random.RandomState = None) -> None:
        """
        Initialize an ExperienceReplayBuffer object.

        Parameters:
        -----------
        buffer_size (int): maximum size of buffer
        batch_size (int): size of each training batch
        seed (int): random seed
        
        """
        self._batch_size = batch_size
        self._buffer_size = buffer_size
        self._buffer = collections.deque(maxlen=buffer_size)
        self._random_state = np.random.RandomState() if random_state is None else random_state
        
    def __len__(self) -> int:
        return len(self._buffer)
    
    @property
    def batch_size(self) -> int:
        return self._batch_size
    
    @property
    def buffer_size(self) -> int:
        return self._buffer_size

    def is_full(self) -> bool:
        return len(self._buffer) == self._buffer_size
    
    def append(self, experience: Experience) -> None:
        """Add a new experience to memory."""
        self._buffer.append(experience)
    
    def sample(self, deleted) -> typing.List[Experience]:
        """Randomly sample a batch of experiences from memory."""
        experiences = []
        idxs = self._random_state.randint(len(self._buffer), size=self._batch_size)
        for i in range(len(idxs)): 
          if (idxs[i] not in deleted):
            experiences.append(self._buffer[idxs[i]])
        # experiences = [self._buffer[idx] for idx in idxs]
        return experiences

In [ ]:
#%%bash

# install required system dependencies
#apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
%pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

%pip install ribs[visualize] gym~=0.17.0 Box2D~=2.3.10
#%pip install pyvirtualdisplay==0.2.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 550.6/550.6 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 KB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyOpenGL-accelerate: filename=PyOpenGL_accelerate-3.1.6-cp38-cp38-linux_x86_64.whl size=2508167 sha256=95f53115696a0716765fd64e8263e816da6d919e8d48b6854fb679275466c378
  Stored in directory: /root/.cache/pip/wheels/ec/04/74/854c06008a26af5a3b8106007e2950b70df3349324239586f2
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This err

In [ ]:
#import pyvirtualdisplay


#_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    #size=(1400, 900))
#_ = _display.start()

In [ ]:
import gym

env = gym.make('LunarLander-v2')
_ = env.seed(42)

In [ ]:
class Agent:
    
    def choose_action(self, state: np.array) -> int:
        """Rule for choosing an action given the current state of the environment."""
        raise NotImplementedError
        
    def save(self, filepath) -> None:
        """Save any important agent state to a file."""
        raise NotImplementedError
        
    def step(self,
             state: np.array,
             action: int,
             reward: float,
             next_state: np.array,
             done: bool) -> None:
        """Update agent's state after observing the effect of its action on the environment."""
        raise NotImplmentedError


In [ ]:
def _train_for_at_most(agent: Agent, env: gym.Env, max_timesteps: int) -> int:
    """Train agent for a maximum number of timesteps."""
    state = env.reset()
    score = 0
    for t in range(max_timesteps):
        action = agent.choose_action(state)   # epislon action generation policy
        next_state, reward, done, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break
    return score

                
def _train_until_done(agent: Agent, env: gym.Env) -> float:
    """Train the agent until the current episode is complete."""
    state = env.reset()
    score = 0
    done = False
    while not done:
        action = agent.choose_action(state)
        next_state, reward, done, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
    return score

import time

def train(agent: Agent,
          env: gym.Env,
          checkpoint_filepath: str,
          target_score: float,
          number_episodes: int,
          maximum_timesteps=None) -> typing.List[float]:
    """
    Reinforcement learning training loop.
    
    Parameters:
    -----------
    agent (Agent): an agent to train.
    env (gym.Env): an environment in which to train the agent.
    checkpoint_filepath (str): filepath used to save the state of the trained agent.
    number_episodes (int): maximum number of training episodes.
    maximum_timsteps (int): maximum number of timesteps per episode.
    
    Returns:
    --------
    scores (list): collection of episode scores from training.
    
    """
    scores = []
    most_recent_scores = collections.deque(maxlen=25)
    for i in range(number_episodes):
        #start = time.time()
        if maximum_timesteps is None:
            score = _train_until_done(agent, env)
        else:
            score = _train_for_at_most(agent, env, maximum_timesteps)         
        scores.append(score)
        most_recent_scores.append(score)
        
        average_score = sum(most_recent_scores) / len(most_recent_scores)
        if average_score >= target_score:
            print(f"\nEnvironment solved in {i:d} episodes!\tAverage Score: {average_score:.2f}")
            agent.save(checkpoint_filepath)
            break
        if (i + 1) % 25 == 0:
            print(f"\rEpisode {i + 1}\tAverage Score: {average_score:.2f}")
        #end = time.time()
        #print(end-start)

    return scores

In [ ]:
import torch
from torch import optim
from torch.nn import functional as F
import random 
import numpy as np
import time
import bisect

from torch import nn
class Net(nn.Module):
  """A non-sparse neural network with four hidden fully-connected layers"""

  def __init__(self,_state_size, number_hidden_units, _action_size):
    super(Net,self).__init__()
    self.input_layer = nn.Linear(_state_size, number_hidden_units, bias=False)
    self.hidden1_layer = nn.Linear(number_hidden_units, number_hidden_units, bias=False)
    self.output_layer = nn.Linear(number_hidden_units, _action_size, bias=False)

  def forward(self, x):
    x = self.input_layer(x)
    x = F.relu(x)
    x = self.hidden1_layer(x)
    x = F.relu(x)
    output = self.output_layer(x)

    return output

class DeepQAgent(Agent):

    def __init__(self,
                 state_size: int,
                 action_size: int,
                 number_hidden_units: int,
                 optimizer_fn: typing.Callable[[typing.Iterable[torch.nn.Parameter]], optim.Optimizer],
                 batch_size: int,
                 buffer_size: int,
                 epsilon_decay_schedule: typing.Callable[[int], float],
                 alpha: float,
                 gamma: float,
                 update_frequency: int,
                 seed: int = None) -> None:
        """
        Initialize a DeepQAgent.
        
        Parameters:
        -----------
        state_size (int): the size of the state space.
        action_size (int): the size of the action space.
        number_hidden_units (int): number of units in the hidden layers.
        optimizer_fn (callable): function that takes Q-network parameters and returns an optimizer.
        batch_size (int): number of experience tuples in each mini-batch.
        buffer_size (int): maximum number of experience tuples stored in the replay buffer.
        epsilon_decay_schdule (callable): function that takes episode number and returns epsilon.
        alpha (float): rate at which the target q-network parameters are updated.
        gamma (float): Controls how much that agent discounts future rewards (0 < gamma <= 1).
        update_frequency (int): frequency (measured in time steps) with which q-network parameters are updated.
        seed (int): random seed
        
        """
        self._state_size = state_size
        self._action_size = action_size
        self._device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # set seeds for reproducibility
        self._random_state = np.random.RandomState() if seed is None else np.random.RandomState(seed)
        if seed is not None:
            torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        
        # initialize agent hyperparameters
        self._experience_replay_buffer = ExperienceReplayBuffer(batch_size, buffer_size, seed)
        self._epsilon_decay_schedule = epsilon_decay_schedule
        self._alpha = alpha
        self._gamma = gamma
        
        # initialize Q-Networks
        self._update_frequency = update_frequency
        self._local_q_network = self._initialize_q_network(number_hidden_units)
        self._target_q_network = self._initialize_q_network(number_hidden_units)
        self._synchronize_q_networks()
        
        # send the networks to the device
        self._local_q_network.to(self._device)
        self._target_q_network.to(self._device)
        
        # initialize the optimizer
        self._optimizer = optimizer_fn(self._local_q_network.parameters())

        # initialize some counters
        self._number_episodes = 0
        self._number_timesteps = 0
        self._number_parameter_updates = 0
        
    def _initialize_q_network(self, number_hidden_units: int) -> nn.Module:
        """Create a neural network for approximating the action-value function."""
        '''
        q_network = nn.Sequential(
            nn.Linear(in_features=self._state_size, out_features=number_hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=number_hidden_units, out_features=number_hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=number_hidden_units, out_features=self._action_size)
        )
        '''
        q_network = Net(self._state_size, number_hidden_units, self._action_size)
        return q_network
        
    def _learn_from(self, experiences: typing.List[Experience]) -> None:
        """Heart of the Deep Q-learning algorithm."""
        states, actions, rewards, next_states, dones = (torch.Tensor(np.array(vs)).to(self._device) for vs in zip(*experiences))
        
        # get max predicted Q values (for next states) from target model
        next_target_q_values, _ = (self._target_q_network(next_states)
                                       .detach()
                                       .max(dim=1))
        
        # compute the new Q' values using the Q-learning formula
        target_q_values = rewards + (self._gamma * next_target_q_values * (1 - dones))
        
        # get expected Q values from local model
        _index = (actions.long()
                         .unsqueeze(dim=1))
        expected_q_values = (self._local_q_network(states)
                                 .gather(dim=1, index=_index))
        # compute the mean squared loss
        loss = F.mse_loss(expected_q_values, target_q_values.unsqueeze(dim=1))
        
        # agent updates the parameters theta of Q using gradient descent
        self._optimizer.zero_grad()
        loss.backward()
        self._optimizer.step()
        
        self._soft_update_target_q_network_parameters()
                 
    def _soft_update_target_q_network_parameters(self) -> None:
        """Soft-update of target q-network parameters with the local q-network parameters."""
        for target_param, local_param in zip(self._target_q_network.parameters(), self._local_q_network.parameters()):
            target_param.data.copy_(self._alpha * local_param.data + (1 - self._alpha) * target_param.data)
    
    def _synchronize_q_networks(self) -> None:
        """Synchronize the target_q_network and the local_q_network."""
        _ = self._target_q_network.load_state_dict(self._local_q_network.state_dict())
           
    def _uniform_random_policy(self, state: torch.Tensor) -> int:
        """Choose an action uniformly at random."""
        return self._random_state.randint(self._action_size)
        
    def _greedy_policy(self, state: torch.Tensor) -> int:
        """Choose an action that maximizes the action_values given the current state."""
        # evaluate the network to compute the action values
        self._local_q_network.eval()
        with torch.no_grad():
            action_values = self._local_q_network(state)
        
        self._local_q_network.train()
        
        # choose the greedy action
        action = (action_values.cpu()  # action_values might reside on the GPU!
                               .argmax()
                               .item())
        return action
    
    def _epsilon_greedy_policy(self, state: torch.Tensor, epsilon: float) -> int:
        """With probability epsilon explore randomly; otherwise exploit knowledge optimally."""
        if self._random_state.random() < epsilon:
            action = self._uniform_random_policy(state)
        else:
            action = self._greedy_policy(state)
        return action

    def choose_action(self, state: np.array) -> int:
        """
        Return the action for given state as per current policy.
        
        Parameters:
        -----------
        state (np.array): current state of the environment.
        
        Return:
        --------
        action (int): an integer representing the chosen action.

        """
        # need to reshape state array and convert to tensor
        state_tensor = (torch.from_numpy(state)
                             .unsqueeze(dim=0)
                             .to(self._device))
        # choose uniform at random if agent has insufficient experience
        if not self.has_sufficient_experience():
            action = self._uniform_random_policy(state_tensor)
        else:
            epsilon = self._epsilon_decay_schedule(self._number_episodes)
            action = self._epsilon_greedy_policy(state_tensor, epsilon)
        return action
    
    def has_sufficient_experience(self) -> bool:
        """True if agent has enough experience to train on a batch of samples; False otherwise."""
        return len(self._experience_replay_buffer) >= self._experience_replay_buffer.batch_size
    
    def save(self, filepath: str) -> None:
        """
        Saves the state of the DeepQAgent.
        
        Parameters:
        -----------
        filepath (str): filepath where the serialized state should be saved.
        
        Notes:
        ------
        The method uses `torch.save` to serialize the state of the q-network, 
        the optimizer, as well as the dictionary of agent hyperparameters.
        
        """
        checkpoint = {
            "q-network-state": self._local_q_network.state_dict(),
            "optimizer-state": self._optimizer.state_dict(),
            "agent-hyperparameters": {
                "alpha": self._alpha,
                "batch_size": self._experience_replay_buffer.batch_size,
                "buffer_size": self._experience_replay_buffer.buffer_size,
                "gamma": self._gamma,
                "update_frequency": self._update_frequency
            }
        }
        torch.save(checkpoint, filepath)
    
    def step(self, state: np.array, action: int, reward: float, next_state: np.array, done: bool) -> None:
        """
        Updates the agent's state based on feedback received from the environment.
        
        Parameters:
        -----------
        state (np.array): the previous state of the environment.
        action (int): the action taken by the agent in the previous state.
        reward (float): the reward received from the environment.
        next_state (np.array): the resulting state of the environment following the action.
        done (bool): True is the training episode is finised; false otherwise.
        
        """
        # save experience in the experience replay buffer
        experience = Experience(state, action, reward, next_state, done)
        self._experience_replay_buffer.append(experience)
            
        if done:
            self._number_episodes += 1
        else:
            self._number_timesteps += 1

            # every so often the agent should learn from experiences
            if self._number_timesteps % self._update_frequency == 0 and self.has_sufficient_experience():
                deleted = self.parameter_prune(1.6) # ATTENTION: ADJUST ALGORITHM & VARIABLES HERE!!!
                #print("Deleted: " + str(len(deleted)))
                #print("Replay Buffer: " + str(len(self._experience_replay_buffer)))
                #print("Ratio: " + str(float(len(deleted))/float(len(self._experience_replay_buffer))))

                '''
                Parameter to Ratios:
                0.1 -> 3%
                1 -> 35%
                1.6 -> 52%
                2.5 -> 70%
                '''
                experiences = self._experience_replay_buffer.sample(deleted)
                self._learn_from(experiences)

    # def interval_prune(self, parameter, percentage):
    #   '''start = time.time()
    #   print("Interval pruning at: " + str(start))'''
    #   average = [0,0,0,0,0,0]
    #   deleted = []
    #   i = 0
    #   while i < min(500, percentage*len(self._experience_replay_buffer)):
    #     etr = self._experience_replay_buffer._buffer[i][0]
    #     average[0] += etr[0]/len(self._experience_replay_buffer)
    #     average[1] += etr[1]/len(self._experience_replay_buffer)
    #     average[2] += etr[2]/len(self._experience_replay_buffer)
    #     average[3] += etr[3]/len(self._experience_replay_buffer)
    #     average[4] += etr[4]/len(self._experience_replay_buffer)
    #     average[5] += etr[5]/len(self._experience_replay_buffer)
    #     i += 1
    #   i = 0
    #   '''firstCompletion = time.time()
    #   print("First loop complete at: " + str(firstCompletion))
    #   print(str(firstCompletion - start))
    #   print(min(500, percentage*len(self._experience_replay_buffer)))'''
    #   while i < len(self._experience_replay_buffer):
    #     etr = self._experience_replay_buffer._buffer[i][0]
    #     distance = ((average[0]-etr[0])**2 + (average[1] - etr[1])**2 + (average[2]-etr[2])**2 + (average[3] - etr[3])**2 + (average[4]-etr[4])**2 + (average[5] - etr[5])**2)**0.5
    #     if(distance > parameter):
    #       deleted.append(i)
    #     i += 1
    #   '''secondCompletion = time.time()
    #   print("Second loop complete at: " + str(secondCompletion))
    #   print(str(secondCompletion - firstCompletion))
    #   print(deleted)'''
    #   return deleted
      
    #     #average = (etr[0][0] + 1)/2) + (etr[0][1] + 0.31)/1.92 + (etr[0][2] + 1.6)/3.3 + (etr[0][3] + 1.96)/2.41 + (etr[0][4] + 3.21)/6.05 + (etr[0][5] + 5.56)/10.84  

    def inverse_threshold_prune(self, parameter):
      i = 0;
      deleted = []
      deleted.clear()
      while i < len(self._experience_replay_buffer): 
          etr = self._experience_replay_buffer._buffer[i]
          if etr[2] < -parameter or etr[2] > parameter:
              deleted.append(i)
          i += 1
      return deleted

    def threshold_prune(self, parameter):
      i = 0;
      deleted = []
      deleted.clear()
      while i < len(self._experience_replay_buffer): 
          etr = self._experience_replay_buffer._buffer[i]
          if etr[2] > -parameter and etr[2] < parameter:
              deleted.append(i)
          i += 1
      return deleted

    def cluster_prune(self, num_clusters, prune_percent):
      deleted = []
      # use proportional pruning based on previous replay buffer distributions (use equal distribution for first pruning)
      # no sorting, only assign an entry to a cluster when it is checked, count how many are pruned from each cluster
      '''
      Reward Based Clustering:
      '''
      cluster_size = 2 / num_clusters
      clusters = [-1 + j*cluster_size for j in range(num_clusters-1)]
      cluster_list = [[] for _ in range(num_clusters + 1)]
      i = 0
      while i < len(self._experience_replay_buffer):
        reward = self._experience_replay_buffer._buffer[i][2]
        cluster_list[bisect.bisect_left(clusters, reward) + 1].append(i)
        i+=1
      for cluster in cluster_list:
        num_to_remove = int(len(cluster) * prune_percent)
        cluster[:] = random.sample(cluster, num_to_remove)
      deleted = sum(cluster_list, [])
      return deleted

      #i=0
      #while i < len(self._experience_replay_buffer):

    def random_prune(self, p):
      deleted = []
      for i in range(len(self._experience_replay_buffer)):
        if np.random.rand() < p:
          deleted.append(i)
      return deleted

In [ ]:
def linear_decay_schedule(episode_number: int,
                          slope: float,
                          minimum_epsilon: float) -> float:
    """Simple linear decay schedule used in the Deepmind paper."""
    return max(1 - slope * episode_number, minimum_epsilon)

def power_decay_schedule(episode_number: int,
                         decay_factor: float,
                         minimum_epsilon: float) -> float:
    """Power decay schedule found in other practical applications."""
    return max(decay_factor**episode_number, minimum_epsilon)

_epsilon_decay_schedule_kwargs = {
    "decay_factor": 0.995,
    "minimum_epsilon": 1e-2,
}
epsilon_decay_schedule = lambda n: power_decay_schedule(n, **_epsilon_decay_schedule_kwargs)

In [ ]:
_optimizer_kwargs = {
    "lr": 1e-2,
    "alpha": 0.99,
    "eps": 1e-08,
    "weight_decay": 0,
    "momentum": 0,
    "centered": False
}
optimizer_fn = lambda parameters: optim.RMSprop(parameters, **_optimizer_kwargs)

In [ ]:
_agent_kwargs = {
    "state_size": env.observation_space.shape[0],
    "action_size": env.action_space.n, 
    "number_hidden_units": 64,
    "optimizer_fn": optimizer_fn,
    "epsilon_decay_schedule": epsilon_decay_schedule,
    "batch_size": 64,
    "buffer_size": 100000,
    "alpha": 1e-3,
    "gamma": 0.99,
    "update_frequency": 4,
    "seed": None,
}
deep_q_agent = DeepQAgent(**_agent_kwargs)

In [ ]:
import matplotlib.pyplot as plt
from IPython import display


def simulate(agent: Agent, env: gym.Env, ax: plt.Axes) -> None:
    state = env.reset()
    img = ax.imshow(env.render(mode='rgb_array'))
    done = False
    while not done:
        action = agent.choose_action(state)
        img.set_data(env.render(mode='rgb_array')) 
        plt.axis('off')
        display.display(plt.gcf())
        display.clear_output(wait=True)
        state, reward, done, _ = env.step(action)       
    env.close()

In [ ]:
'''
import matplotlib.pyplot as plt
def prune(deep_q, parameter):
  i = 0;
  deleted = []
  indexes = []
  rewards = []
  while i < len(deep_q._experience_replay_buffer):
      etr = deep_q._experience_replay_buffer._buffer[i]
      i += 1 
      indexes.append(i)
      rewards.append(etr[2])
      if etr[2] > -parameter and etr[2] < parameter:
          deleted.append(i)
  plt.scatter(indexes, rewards)
  plt.show()
  return deleted

print(prune(deep_q_agent, 0.1))
'''

'''
def prune(deep_q):
  
'''



       # Compare all entries to that average; if they are too far, they are outliers and can be removed UNLESS r > x
      # Option Four: Threshold
      # r > x OR r < y, keep it, otherwise delete [s, a, s1, r]
      # Option One: Average
      # Take average entry data (s, a, s1, r)
      # Option Two: Sort
      # Sort all entries from least to greatest (normalization?)
      # Measure intervals between each entry: if the interval is too small, then they are too close to each other and one can be removed (the one farther away from the next two closest)
      # Option Three: Probability
      # Somehow determine a way to check probability of each state occurring (maybe if the coordinates are very far left/right, etc)
      # Quantify this probability and remove the lowest probability ones (continuous -> discrete)

'\ndef prune(deep_q):\n  \n'

In [ ]:
#_, ax = plt.subplots(1, 1, figsize=(10, 8))
#simulate(deep_q_agent, env, ax)

In [ ]:
scores = train(deep_q_agent, env, "checkpoint.pth", number_episodes=300, target_score=200)

Episode 25	Average Score: -182.71
Episode 50	Average Score: -140.10
Episode 75	Average Score: -151.25
Episode 100	Average Score: -135.94
Episode 125	Average Score: -121.05
Episode 150	Average Score: -252.09
Episode 175	Average Score: -167.85
Episode 200	Average Score: -200.88
Episode 225	Average Score: -163.21
Episode 250	Average Score: -130.13
Episode 275	Average Score: -146.13
Episode 300	Average Score: -137.91
Episode 325	Average Score: -129.28


KeyboardInterrupt: ignored

In [ ]:
#_, ax = plt.subplots(1, 1, figsize=(10, 8))
#simulate(deep_q_agent, env, ax)